In [1]:
import pandas as pd
import json
from openai import OpenAI
import ast
import re

Test data

In [5]:
data_path = r"C:\Users\btuud\Downloads\Component_generator\train_langflow_custom_component_without_python_code.xlsx"
df = pd.read_excel(data_path)
print(df)

    Id                 Name Difficulty                      Category  \
0    1   Sum of two numbers       Easy                          Math   
1    2                 Echo       Easy            Message Processing   
2    3           Chat Input     Medium               Chat Components   
3    4           Text Input       Easy            Message Processing   
4    5          Chat Output     Medium               Chat Components   
5    6          Text Output       Easy            Message Processing   
6    7   Sub of two numbers       Easy                          Math   
7    8  Sum of three number       Easy                          Math   
8    9                  URL       Hard                  Web Scraping   
9   10                 File     Medium              Document Loading   
10  11               Prompt       Easy                        Prompt   
11  12           Parse Data     Meduim               Data Processing   
12  13      Data to Message     Meduim               Data Proces

2) Add message, content, and roles

In [38]:
def convert_to_training_format(input_file, output_file):
    # Read the input JSONL file
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    training_examples = []
    
    for line in lines:
        try:
            # Parse the line as JSON array
            data = json.loads(line.strip())
            
            # Extract the components from each line
            name = data[0]  # e.g., "Sum of two numbers"
            component_json = data[1]  # The component JSON string
            description = data[2]  # e.g., "Takes two inputs and returns their sum..."
            num_input = data[3]  # e.g., "2"
            input_type = data[4]  # e.g., "Message"
            num_output = data[5]  # e.g., "1"
            output_type = data[6]  # e.g., "Message"
            
            # Create the prompt
            prompt = f"Generate a LangFlow custom component in JSON format that {description}. "
            prompt += f"The component should have {num_input} {input_type} input"
            prompt += "s" if int(num_input) > 1 else ""
            prompt += f" and {num_output} {output_type} output"
            prompt += "s" if int(num_output) > 1 else ""
            prompt += "."
            
            # Create the training example in the correct format
            training_example = {
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that generates LangFlow custom components in JSON format based on user descriptions."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    },
                    {
                        "role": "assistant",
                        "content": component_json
                    }
                ]
            }
            
            training_examples.append(training_example)
            print(f"Successfully converted example: {name}")
            
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")
            continue
        except Exception as e:
            print(f"Error processing line: {e}")
            continue
    
    # Write the converted examples to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_examples:
            f.write(json.dumps(example) + '\n')
    
    return len(training_examples)


In [39]:
input_file = "tableConvert.com_yogn21.jsonl"
output_file = "trainning.jsonl"

print("Starting conversion...")
num_converted = convert_to_training_format(input_file, output_file)
print(f"\nConversion complete! Successfully converted {num_converted} examples.")
print(f"Output saved to: {output_file}")

Starting conversion...
Successfully converted example: Sum of two numbers
Successfully converted example: Echo
Successfully converted example: Chat Input
Successfully converted example: Text Input
Successfully converted example: Chat Output
Successfully converted example: Text Output
Successfully converted example: Sub of two numbers
Successfully converted example: Sum of three number
Successfully converted example: URL
Successfully converted example: File
Successfully converted example: Prompt
Successfully converted example: Parse Data
Successfully converted example: Data to Message
Successfully converted example: Current Date
Error processing line: invalid literal for int() with base 10: 'MessageTextInput (Supports manual entry or refresh for a new ID)"'
Successfully converted example: Tavily Seach
Successfully converted example: Youtube Playlist
Successfully converted example: YouTube Channel
Successfully converted example: YouTube Transcript
Successfully converted example: YouTube 

3) Fine-tune gpt-4o-mini

3.1 Get id for uploading file to openai

In [ ]:
client = OpenAI(api_key = "")
client.files.create(
    file = open("train_text.jsonl", "rb"),
    purpose="fine-tune"
)

FileObject(id='file-Y7gZdxKKcgK4YYZJM2BfJ7', bytes=69065, created_at=1739429403, filename='train_text.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

3.2 Give Fine-tune job to openai

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-Y7gZdxKKcgK4YYZJM2BfJ7",
  model="gpt-4o-mini-2024-07-18"
) 

FineTuningJob(id='ftjob-IW7j1BQZglhxaUGJkeIBgCFF', created_at=1739429412, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1242169928, status='validating_files', trained_tokens=None, training_file='file-Y7gZdxKKcgK4YYZJM2BfJ7', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [ ]:
# Replace with your actual fine-tuning job ID
fine_tune_job_id = ""

# Retrieve job status
job_status = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
print(job_status)
#ft:gpt-4o-mini-2024-07-18:personal::AzxXDnBX


FineTuningJob(id='ftjob-IW7j1BQZglhxaUGJkeIBgCFF', created_at=1739429412, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1242169928, status='validating_files', trained_tokens=None, training_file='file-Y7gZdxKKcgK4YYZJM2BfJ7', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 'auto', 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}})


In [117]:
job_id = "ftjob-IW7j1BQZglhxaUGJkeIBgCFF"
job_info = client.fine_tuning.jobs.retrieve(job_id)

print(job_info)

FineTuningJob(id='ftjob-IW7j1BQZglhxaUGJkeIBgCFF', created_at=1739429412, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1242169928, status='running', trained_tokens=None, training_file='file-Y7gZdxKKcgK4YYZJM2BfJ7', validation_file=None, estimated_finish=1739429872, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 5, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}})


In [ ]:
from openai import OpenAI

client = OpenAI()

prompt = '''Generate a LangFlow custom component in JSON format that multiplies 2 numbers. 
The component should have 2 Message inputs and 1 Message output.

Here is an example JSON format for an Echo function:

{"data":{"edges":[],"nodes":[{"data":{"node":{"template":{"_type":"Component","code":{"type":"code","required":true,"placeholder":"","list":false,"show":true,"multiline":true,"value":"# from langflow.field_typing import Data\nfrom langflow.custom import Component\nfrom langflow.io import MessageTextInput, Output\nfrom langflow.schema import Data\n\n\nclass CustomComponent(Component):\n    display_name = \"Custom Component\"\n    description = \"Use as a template to create your own component.\"\n    documentation: str = \"http://docs.langflow.org/components/custom\"\n    icon = \"code\"\n    name = \"CustomComponent\"\n\n    inputs = [\n        MessageTextInput(\n            name=\"input_value\",\n            display_name=\"Input Value\",\n            info=\"This is a custom component Input\",\n            value=\"Hello, World!\",\n            tool_mode=True,\n        ),\n    ]\n\n    outputs = [\n        Output(display_name=\"Output\", name=\"output\", method=\"build_output\"),\n    ]\n\n    def build_output(self) -> Data:\n        data = Data(value=self.input_value)\n        self.status = data\n        return data\n","fileTypes":[],"file_path":"","password":false,"name":"code","advanced":true,"dynamic":true,"info":"","load_from_db":false,"title_case":false},"input_value":{"tool_mode":true,"trace_as_input":true,"trace_as_metadata":true,"load_from_db":false,"list":false,"required":false,"placeholder":"","show":true,"name":"input_value","value":"Hello, World!","display_name":"Input Value","advanced":false,"input_types":["Message"],"dynamic":false,"info":"This is a echo Message Input","title_case":false,"type":"str","_input_type":"MessageTextInput"}},"description":"Use as a template to create your own component.","icon":"code","base_classes":["Data"],"display_name":"Echo","documentation":"http://docs.langflow.org/components/custom","custom_fields":{},"output_types":[],"pinned":false,"conditional_paths":[],"frozen":false,"outputs":[{"types":["Data"],"selected":"Data","name":"output","display_name":"Output","method":"build_output","value":"__UNDEFINED__","cache":true}],"field_order":["input_value"],"beta":false,"legacy":false,"edited":false,"metadata":{},"tool_mode":false,"lf_version":"1.1.1","official":false},"type":"CustomComponent","id":"CustomComponent-EmZ4A"},"id":"CustomComponent-EmZ4A","position":{"x":0,"y":0},"type":"genericNode"}],"viewport":{"x":1,"y":1,"zoom":1}},"description":"Echo user input","name":"Echo","id":"CustomComponent-EmZ4A","is_component":true,"last_tested_version":"1.1.1"}

Please generate a similar JSON structure for a component that multiplies two numbers.
'''

response = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::B0NSj8Or",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)


Here is a LangFlow custom component for multiplying two numbers, represented in JSON format:

```json
{
  "data": {
    "edges": [],
    "nodes": [
      {
        "data": {
          "node": {
            "template": {
              "_type": "Component",
              "code": {
                "type": "code",
                "required": true,
                "placeholder": "",
                "list": false,
                "show": true,
                "multiline": true,
                "value": "# from langflow.field_typing import Data\nfrom langflow.custom import Component\nfrom langflow.io import MessageTextInput, Output\nfrom langflow.schema import Data\n\n\nclass CustomComponent(Component):\n    display_name = \"Custom Component\"\n    description = \"Use as a template to create your own component.\"\n    documentation: str = \"http://docs.langflow.org/components/custom\"\n    icon = \"code\"\n    name = \"CustomComponent\"\n\n    inputs = [\n        MessageTextInput(\n          